# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from gkey import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
data_path = "City_Data/City_Data2.csv"
df = pd.read_csv(data_path)
df.head(10)

,Unnamed: 0,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Avarua,56,CK,78,-21.21,-159.78,27.00,8.20
1,1,Tiksi,94,RU,86,71.69,128.87,-25.31,4.11
2,2,Mataura,98,NZ,53,-46.19,168.86,17.22,6.26
3,3,Vardo,100,NO,86,70.37,31.11,-2.29,9.30
4,4,Dubbo,100,AU,64,-32.25,148.62,21.00,7.70
5,5,Castro,100,BR,96,-24.79,-50.01,21.11,2.17
6,6,Hermanus,0,ZA,84,-34.42,19.23,20.25,1.79
7,7,Saint-Philippe,20,RE,73,-21.36,55.77,20.67,3.10
8,8,Khovd,96,MN,98,48.01,91.64,-20.82,0.33
9,9,Kapaa,75,US,73,22.08,-159.32,23.95,3.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# Access maps with API key
gmaps.configure(api_key=gkey)
lat_lng_df = df[['Lat','Lng']]
lat_lng_df.head()

,Lat,Lng
0,-21.21,-159.78
1,71.69,128.87
2,-46.19,168.86
3,70.37,31.11
4,-32.25,148.62


In [43]:
# Customize the size of the figure
fig = gmaps.figure()

In [44]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(lat_lng_df)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
df.head()

,Unnamed: 0,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Avarua,56,CK,78,-21.21,-159.78,27.00,8.20
1,1,Tiksi,94,RU,86,71.69,128.87,-25.31,4.11
2,2,Mataura,98,NZ,53,-46.19,168.86,17.22,6.26
3,3,Vardo,100,NO,86,70.37,31.11,-2.29,9.30
4,4,Dubbo,100,AU,64,-32.25,148.62,21.00,7.70


In [46]:
# filted the database
df.head()
filted_df = df.loc[(df['Max Temp'] > 20) & (df['Max Temp'] < 25), :]
filted_df = filted_df.loc[(filted_df['Cloudiness'] < 80), :]
filted_df = filted_df.loc[(filted_df['Wind Speed'] > 6) & (filted_df['Wind Speed'] < 8), :]
filted_df.count()

Unnamed: 0    6
City          6
Cloudiness    6
Country       6
Humidity      6
Lat           6
Lng           6
Max Temp      6
Wind Speed    6
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
hotel_df = filted_df.reset_index().drop(columns=['Unnamed: 0',
                                   'Cloudiness',
                                   'Humidity',
                                   'Max Temp',
                                   'Wind Speed',
                                   'index'
                                  ])

In [71]:
hotel_df['Coordinate'] = hotel_df['Lat'].map(str) + ',' + hotel_df['Lng'].map(str)
hotel_df

,City,Country,Lat,Lng,Coordinate
0,Vila do Maio,CV,15.13,-23.22,"15.13,-23.22"
1,Kahului,US,20.89,-156.47,"20.89,-156.47"
2,Praia,CV,14.92,-23.51,"14.92,-23.51"
3,Antofagasta,CL,-23.65,-70.40,"-23.65,-70.4"
4,Juchipila,MX,21.42,-103.12,"21.42,-103.12"
5,Sittwe,MM,20.15,92.90,"20.15,92.9"


In [72]:
# set parameter dictionary
hotel_df['Hotel Name'] = ""

for i in hotel_df.index:
    params = {
        "location": hotel_df.loc[i, 'Coordinate'],
        "types": 'hotel',
        "rankby": "distance",
        "key": gkey
        }
# Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Run request
    response = requests.get(base_url, params)
    
    my_hotel = response.json()
    hotel_df.loc[i, 'Hotel Name'] = (my_hotel["results"][0]["name"])
    print(my_hotel["results"][0]["name"])
    

Forte de São José
Hawaii Community Foundation
Atlantico hotelaria e lar Lda
Overmine Ltda.
Unidad Deportiva Juchipila Zacatecas
ရခိုင်မယ်အိမ်


In [73]:
hotel_df

,City,Country,Lat,Lng,Coordinate,Hotel Name
0,Vila do Maio,CV,15.13,-23.22,"15.13,-23.22",Forte de São José
1,Kahului,US,20.89,-156.47,"20.89,-156.47",Hawaii Community Foundation
2,Praia,CV,14.92,-23.51,"14.92,-23.51",Atlantico hotelaria e lar Lda
3,Antofagasta,CL,-23.65,-70.40,"-23.65,-70.4",Overmine Ltda.
4,Juchipila,MX,21.42,-103.12,"21.42,-103.12",Unidad Deportiva Juchipila Zacatecas
5,Sittwe,MM,20.15,92.90,"20.15,92.9",ရခိုင်မယ်အိမ်


In [77]:
hotel_df.to_csv('hotel_infos.csv')

In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [97]:
weights = df['Humidity']/20

In [98]:
# Plot Map
fig = gmaps.figure(
    layout={
        'width': '500px',
        'height': '800px'
    }
)

# Create and add heat layer
heat_layer = gmaps.heatmap_layer(
    lat_lng_df,
    weights=weights, 
    max_intensity=max(weights),
    point_radius=10)
fig.add_layer(heat_layer)

# Create and add marker layer
markers = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='800px', width='500px'))